In [1]:
%matplotlib notebook

import os
import sys
import cv2
import matplotlib.pyplot as plt
import numpy as np
from darkflow.net.build import TFNet
import track.sort.sort as srt

In [2]:
# print(os.getcwd())
# os.chdir('..')
# %run store_results.py

In [3]:
# directory with images from video
working_directory = '/home/analyticsuser/data/Husband Surprises Wife by Filling House With Puppies!'
#working_directory = '/home/analyticsuser/data/Nest Cam - Owner catches burglars in act on her phone'
#working_directory = '/data01/OC_RESINET/process/ResiIndoor03-YT/Indoor-family2'

# only images
files_in_directory = sorted([os.path.join(working_directory, file_name)
                      for file_name in os.listdir(working_directory)
                      if os.path.splitext(file_name)[1].lower() in ('.jpg', '.jpeg')])

# colors for different bboxes for visualization
colors = [(255,0,0), (0,255,0),(0,0,255),(255,255,0),(0,255,255),(255,0,255),(192,192,192),(128,128,128),
          (128,0,0),(128,128,0),(0,128,0),(128,0,128),(0,128,128),(0,0,128),(0,0,0)]

# configuration for YOLO
options = {'model': '/home/analyticsuser/darkflow/cfg/yolo.2.0.cfg', 
           'load': '/home/analyticsuser/darkflow/cfg/yolo.2.0.weights', 'threshold': 0.1}
# options = {'model': '/home/analyticsuser/darkflow/cfg/tiny-yolo.cfg', 
#            'load': '/home/analyticsuser/darkflow/cfg/tiny-yolo.weights', 'threshold': 0.1}

tfnet = TFNet(options)

# initialize SORT tracker:
# mot_tracker = srt.Sort(10,1,5,0.8,0.94,0.8,2.0)
# - 10 consecutive frames with no detection to terminate an existing track
# - 1 consecutive frame (i.e. after an initial frame) to activate the new track (before that will not produce any output)
# - 5 frames to extrapolate an active track (i.e. show predicted output eventhough no detection present for this track)
# - 0.8 unassigned detection over track bbox overlap (intersection over union) threshold to eliminate the detection
# - 0.94 unassigned detection over another stronger unassigned detection overlap (intersection over self bbox area) threshold to eliminate the detection
# - 0.8 unassigned detection over another stronger unassigned detection overlap (intersection over larger bbox area) threshold to eliminate the detection
# - 2.0 defines "stronger" detection disregarding the size -- if the score of the unassigned detection overlapping 
#       another unassigned detection is 2.0-times higher than the score of the other detection, it is stronger, otherwise 
#       larger size decides
mot_tracker = srt.Sort(10,0,10,0.55,0.90,0.6,2.0)

Parsing ./cfg/yolo.2.0.cfg
Parsing /home/analyticsuser/darkflow/cfg/yolo.2.0.cfg
Loading /home/analyticsuser/darkflow/cfg/yolo.2.0.weights ...
Successfully identified 269862452 bytes
Finished in 0.04532146453857422s


In [4]:
def visualize_rectangle(imgcv,detection,colors,idx):
    # detected rectangle
    cv2.rectangle(imgcv, (detection['topleft']['x'], detection['topleft']['y']), 
                  (detection['bottomright']['x'], detection['bottomright']['y']), 
                  colors[idx % len(colors)], 10)
    # gray background bellow text
    cv2.rectangle(imgcv, (detection['topleft']['x'], detection['topleft']['y'] - 20), 
                  (detection['bottomright']['x'], detection['topleft']['y']), (125, 125, 125), -1)
    # text class:confidence
    cv2.putText(imgcv, detection['label'] + ' : %.2f' % detection['confidence'], 
                (detection['topleft']['x'] + 5, detection['topleft']['y'] - 7), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)

    
##########################################################
# split detection on stacked image into upper/lower view,     
# reduce the size of upper bbox when overlaid to bottom view
# set new coordinates to lower view (remove height/2)
def split_detection(imgcv, result):  
    detection1 = list([])
    detection2 = list([])
    height,width,cspace = imgcv.shape
    
    #result_filter = list(filter(lambda x: x['label'] == 'person' or x['label'] == 'dog' or x['label'] == 'cat',result))
    result_filter = list(filter(lambda x: x['label'] == 'person',result))
    detection1 = list(filter(lambda x: x['topleft']['y'] < height/2,result_filter))
    detection2 = list(filter(lambda x: x['topleft']['y'] >= height/2,result_filter))
    # for detection1 we have to limit the height
    for d in detection1:
        #print([height/2, d['bottomright']['y']])
        d['bottomright']['y'] = int(np.min([height/2, d['bottomright']['y']]))
    
    # for detection2 we need to substract the height/2
    #detection2 = list(map(lambda x: float(x['topleft']['y']) - height/2, detection2))
    for d in detection2:
        #print(d)
        d['topleft']['y'] -= int(height/2)
        d['bottomright']['y'] -= int(height/2)

    return detection1, detection2

In [5]:
rootDir = '/data01/OC_RESINET/process/ResiIndoor05-Pond5'

for iFolder in os.listdir(rootDir):
    working_directory = os.path.join(rootDir, iFolder)
    print(working_directory)
    files_in_directory = sorted([os.path.join(working_directory, file_name)
                  for file_name in os.listdir(working_directory)
                  if os.path.splitext(file_name)[1].lower() in ('.jpg', '.jpeg')])
    
    # add extra frame in case the number of frames is odd
    files_in_directory.extend(files_in_directory[-1:])
    #print(files_in_directory)
    
    # create CSV
    # init a dataframe with raw detections
    detections_info_file = 'Yolo_v2_0.1_stacked_images' + '.csv'
    with open(os.path.join(working_directory,  detections_info_file), 'w') as output:
        output.write('Filename, TopLeftX,TopLeftY,BottomRightX,BottomRightY,Confidence,crop_TopLeftX,crop_TopLeftY,crop_BottomRightX,crop_BottomRightY\n')    

    tracking_info_file = 'Yolo_v2_0.1_SORT_stacked_images' + '.csv'
    with open(os.path.join(working_directory,  tracking_info_file), 'w') as output:
        output.write('Filename, ID, TopLeftX,TopLeftY,BottomRightX,BottomRightY,Confidence,crop_TopLeftX,crop_TopLeftY,crop_BottomRightX,crop_BottomRightY\n')    

################################################################
# loop over JPGS

    for iFile,file_name in enumerate(files_in_directory[::1]):
        #print(file_name)
        ii = int(os.path.splitext(file_name)[0][-5:])
        # save first frame
        if  np.remainder(ii,2) > 0:
            imgcv1 = cv2.imread(file_name)
            file_name1 = os.path.basename(file_name)
        # save second frame and process
        else :
            imgcv2 = cv2.imread(file_name)
            imgcv = np.vstack([imgcv1, imgcv2])
            img1 = imgcv.copy()
            file_name2 = os.path.basename(file_name)
            ######################################
            # RUN YOLO on vertically stacked images
            result = tfnet.return_predict(imgcv)
            #print(type(result))

            # separate results to upper/lower view
            detection1, detection2 = split_detection(imgcv, result)
            #print(['detection1 = ',detection1])
            #print(['detection2 = ', detection2])


            ######################################
            # continue with tracking
            dets01 = list(map( lambda x: [float(x['topleft']['x']),float(x['topleft']['y']),
                                         float(x['bottomright']['x']),float(x['bottomright']['y']),
                                         x['confidence']], detection1))
            dets1 = np.asarray( dets01 )
            objs1, termtracks1 = mot_tracker.update(dets1)        

            dets02 = list(map( lambda x: [float(x['topleft']['x']),float(x['topleft']['y']),
                                         float(x['bottomright']['x']),float(x['bottomright']['y']),
                                         x['confidence']], detection2))        
            dets2 = np.asarray( dets02 )
            objs2, termtracks2 = mot_tracker.update(dets1)                

            ######################################            
            # save the DETECTION results to CSV
            with open(os.path.join(working_directory, detections_info_file), 'a') as output_file:
                for bbo_f in dets1:
                    #print('bbo_f: ', bbo_f )
                    bbo = list(map(lambda x: int(round(float(x))), bbo_f[0:4]))
                    obsc = bbo_f[4]                    
                    #output.write('Filename, TopLeftX,TopLeftY,BottomRightX,BottomRightY,Confidence,crop_TopLeftX,crop_TopLeftY,crop_BottomRightX,crop_BottomRightY\n')    
                    output_file.write(f"{file_name1},{bbo[0]},{bbo[1]},{bbo[2]},{bbo[3]},{obsc},{'0'},{'0'},{'0'},{'0'}\n")
                if file_name1 != file_name2:
                    for bbo_f in dets2:
                        #print('bbo_f: ', bbo_f )
                        bbo = list(map(lambda x: int(round(float(x))), bbo_f[0:4]))
                        obsc = bbo_f[4]                    
                        #output.write('Filename, TopLeftX,TopLeftY,BottomRightX,BottomRightY,Confidence,crop_TopLeftX,crop_TopLeftY,crop_BottomRightX,crop_BottomRightY\n')    
                        output_file.write(f"{file_name2},{bbo[0]},{bbo[1]},{bbo[2]},{bbo[3]},{obsc},{'0'},{'0'},{'0'},{'0'}\n")
                    
                    
            # save the TRACKING results to CSV
            with open(os.path.join(working_directory,  tracking_info_file), 'a') as output:
            #output.write('Filename, ID, TopLeftX,TopLeftY,BottomRightX,BottomRightY,Confidence,crop_TopLeftX,crop_TopLeftY,crop_BottomRightX,crop_BottomRightY\n')                    
                for bbo_f in objs1:
                    #print('bbo_f: ', bbo_f )
                    bbo = list(map(lambda x: int(round(float(x))), bbo_f[0:4]))
                    obid = int(bbo_f[5])
                    obsc = bbo_f[4]
                    output.write(f"{file_name1},{obid},{bbo[0]},{bbo[1]},{bbo[2]},{bbo[3]},{obsc},{'0'},{'0'},{'0'},{'0'}\n")

                if file_name1 != file_name2:
                    for bbo_f in objs1:
                        #print('bbo_f: ', bbo_f )
                        bbo = list(map(lambda x: int(round(float(x))), bbo_f[0:4]))
                        obid = int(bbo_f[5])
                        obsc = bbo_f[4]
                        output.write(f"{file_name2},{obid},{bbo[0]},{bbo[1]},{bbo[2]},{bbo[3]},{obsc},{'0'},{'0'},{'0'},{'0'}\n")




/data01/OC_RESINET/process/ResiIndoor05-Pond5/indoor_022
/data01/OC_RESINET/process/ResiIndoor05-Pond5/indoor_door_005
/data01/OC_RESINET/process/ResiIndoor05-Pond5/front_door_014
/data01/OC_RESINET/process/ResiIndoor05-Pond5/indoor_025
/data01/OC_RESINET/process/ResiIndoor05-Pond5/indoor_door_012
/data01/OC_RESINET/process/ResiIndoor05-Pond5/indoor_053
/data01/OC_RESINET/process/ResiIndoor05-Pond5/indoor_002_falseAlarm
/data01/OC_RESINET/process/ResiIndoor05-Pond5/indoor_043
/data01/OC_RESINET/process/ResiIndoor05-Pond5/indoor_door_010
/data01/OC_RESINET/process/ResiIndoor05-Pond5/indoor_018
/data01/OC_RESINET/process/ResiIndoor05-Pond5/indoor_028
/data01/OC_RESINET/process/ResiIndoor05-Pond5/indoor_024
/data01/OC_RESINET/process/ResiIndoor05-Pond5/front_door_011
/data01/OC_RESINET/process/ResiIndoor05-Pond5/indoor_021
/data01/OC_RESINET/process/ResiIndoor05-Pond5/indoor_020
/data01/OC_RESINET/process/ResiIndoor05-Pond5/front_door_006
/data01/OC_RESINET/process/ResiIndoor05-Pond5/fron